In [ ]:
from fastai.vision import *
from fastai.metrics import accuracy

## Data

In [ ]:
# Copy for FastAI
!mkdir -p data
!cp -R ../input/dsnet-kaggledays-hackathon/train/train data/train
# !cp -R ../input/dsnet-kaggledays-hackathon/test/test data/test
!cp ../input/dsnet-kaggledays-hackathon/sample_submission.csv data/sample_submission.csv

In [ ]:
!ls data

In [ ]:
DATA_DIR = Path('data')

In [ ]:
?Path

In [ ]:
DATA_DIR/'train'

In [ ]:
bs = 64

In [ ]:
data = ImageDataBunch.from_folder(DATA_DIR, valid_pct=0.2, ds_tfms=get_transforms(flip_vert="True"), bs=180, size=64)

In [ ]:
?get_tranforms()

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

## Model

In [ ]:
learn = cnn_learner(data, models.resnet18, pretrained=False, metrics=accuracy)

In [ ]:
learn.model

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, 3e-3)

In [ ]:
# Save the model
learn.save('simple-model')

In [ ]:
# Load the model
learn.load('simple-model');

In [ ]:
!cp -R ../input/dsnet-kaggledays-hackathon/test/test data/test

## Inference

In [ ]:
# Load submission file
sample_df = pd.read_csv(DATA_DIR/'sample_submission.csv')
sample_df.head()

In [ ]:
# Generate test predictions
learn.data.add_test(ImageList.from_df(sample_df,DATA_DIR,folder='test'))

In [ ]:
# Load up submission file
preds,y = learn.get_preds(DatasetType.Test)

In [ ]:
# Convert predictions to classes
pred_classes = [data.classes[c] for c in list(preds.argmax(dim=1).numpy())]
pred_classes[:10]

In [ ]:
# Add the prediction
sample_df.predicted_class = pred_classes
sample_df.head()

In [ ]:
# Save the submission file
sample_df.to_csv('submission.csv',index=False)

from IPython.display import FileLink
FileLink('submission.csv')

In [ ]:
# Clean up (for commit)
!cp -R data/models models # Move the models out
!rm -rf data # Delete the data